In [24]:
# Cell 1: All Imports and Setup
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks.base import BaseCallbackHandler

pd.set_option("display.max_colwidth", None)  # show full text inside cells
pd.set_option("display.max_columns", None)   # show all columns
pd.set_option("display.max_rows", None)      # show all rows


# Load environment variables from the .env file in the src folder
load_dotenv(dotenv_path="src/.env")

# Get the API key and initialize the LLM
key = os.getenv("GROQ_API_KEY")
if not key:
    raise ValueError("GROQ_API_KEY not found. Make sure it's set in your src/.env file.")

llm = ChatGroq(groq_api_key=key, model_name="llama3-70b-8192", temperature=0.5)

print("Setup Complete: LLM is initialized.")
print(f"Using key: ...{key[-4:]}") # Print last 4 chars for verification

Setup Complete: LLM is initialized.
Using key: ...5WI0


In [25]:
from langchain.callbacks.base import BaseCallbackHandler

class GroqCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.request_count = 0

    def on_llm_start(self, serialized, prompts, **kwargs):
        self.request_count += 1
        print(f"Request #{self.request_count} started with prompts: {prompts}")

    def on_llm_end(self, response, **kwargs):
        print(f"Request #{self.request_count} finished with response: {response}\n")

    def on_llm_error(self, error, **kwargs):
        print(f"Request #{self.request_count} encountered an error: {error}\n")


In [26]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CE267C4E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CE267C5810>, model_name='llama3-70b-8192', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [27]:
RESPONSE_JSON = {
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "2": {
        "no": "2",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "3": {
        "no": "3",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    }
    
    
}

In [28]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, create a quiz of exactly {number} unique multiple choice questions for {subject} students in {tone} tone.
Each question must have 4 options (a, b, c, d) and a correct answer.
You must output only valid JSON matching the RESPONSE_JSON schema below. Do not output anything else. If you cannot, return an error message.
### RESPONSE_JSON
{response_json}
"""

In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [30]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [31]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, create a quiz of exactly {number} unique multiple choice questions for {subject} students in {tone} tone.\nEach question must have 4 options (a, b, c, d) and a correct answer.\nYou must output only valid JSON matching the RESPONSE_JSON schema below. Do not output anything else. If you cannot, return an error message.\n### RESPONSE_JSON\n{response_json}\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CE267C4E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CE267C5810>, model_name='llama3-70b-8192', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********')), output_key='quiz', output_parser=StrOutputParser(), llm_kwargs={})

In [32]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [33]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
    
)

In [34]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [35]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [36]:
file_path=r"C:\Users\Sai Sanjith\mcqgenerator\data.txt"

In [37]:
with open (file_path, "r") as file:
    text= file.read()

In [38]:
print(text)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical viewpoint, probably approximately c

In [39]:
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [41]:
# Run the chain and print the results
handler = GroqCallbackHandler()
response = generate_evaluate_chain(
    {
        "text": text,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    },
    callbacks=[handler]
)
print("Quiz Output:\n", response.get("quiz"))
print("\nReview Output:\n", response.get("review"))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory 

In [42]:
quiz=response.get("quiz")

In [43]:


import json
import re

print("Raw quiz output:", repr(quiz))

if quiz:
    # Try to extract the JSON block from the output
    match = re.search(r'(\{[\s\S]*\})', quiz)
    if match:
        quiz_json_str = match.group(1)
        try:
            quiz_json = json.loads(quiz_json_str)
            for q_no, q_data in quiz_json.items():
                print(f"\nQuestion {q_no}: {q_data.get('mcq', '')}")
                options = q_data.get('options', {})
                for opt, val in options.items():
                    print(f"  {opt}. {val}")
                print(f"Correct Answer: {q_data.get('correct', '')}")
        except Exception as e:
            print("Could not parse extracted JSON block.", e)
    else:
        print("Could not find a JSON block in the quiz output.")
else:
    print("Quiz output is empty.")

Raw quiz output: 'Here are 5 multiple choice questions for data science students based on the provided text:\n\n{\n"1": {\n"no": "1",\n"mcq": "What is the primary goal of machine learning?",\n"options": {\n"a": "To achieve artificial intelligence",\n"b": "To classify data based on models",\n"c": "To make predictions for future outcomes",\n"d": "All of the above"\n},\n"correct": "d"\n},\n"2": {\n"no": "2",\n"mcq": "What is the connection between machine learning and data compression?",\n"options": {\n"a": "Machine learning is used to compress data",\n"b": "Data compression is used to improve machine learning",\n"c": "A system that predicts posterior probabilities can be used for optimal data compression",\n"d": "There is no connection between machine learning and data compression"\n},\n"correct": "c"\n},\n"3": {\n"no": "3",\n"mcq": "What is the main difference between machine learning and data mining?",\n"options": {\n"a": "Machine learning focuses on prediction, while data mining focus

In [44]:
import json
import re
import pandas as pd

quiz_table_data = []

print("Raw quiz output:", repr(quiz))

if quiz:
    # Try to extract the JSON block from the output
    match = re.search(r'(\{[\s\S]*\})', quiz)
    if match:
        quiz_json_str = match.group(1)
        try:
            quiz_json = json.loads(quiz_json_str)

            # Build structured table
            for q_no, q_data in quiz_json.items():
                options_str = " | ".join([f"{opt}. {val}" for opt, val in q_data.get("options", {}).items()])
                
                quiz_table_data.append([
                    q_no,
                    q_data.get("mcq", ""),
                    options_str,
                    q_data.get("correct", "")
                ])

            # Create DataFrame
            df = pd.DataFrame(quiz_table_data,
                              columns=["Question Number", "Question", "Options", "Correct Answer"])

            # Display DataFrame with alignment in Jupyter
            from IPython.display import display
            display(df)

            # Save DataFrame to CSV
            df.to_csv('quiz.csv', index=False)
            print('Quiz saved to quiz.csv')

        except Exception as e:
            print("Could not parse extracted JSON block.", e)
    else:
        print("Could not find a JSON block in the quiz output.")
else:
    print("Quiz output is empty.")


Raw quiz output: 'Here are 5 multiple choice questions for data science students based on the provided text:\n\n{\n"1": {\n"no": "1",\n"mcq": "What is the primary goal of machine learning?",\n"options": {\n"a": "To achieve artificial intelligence",\n"b": "To classify data based on models",\n"c": "To make predictions for future outcomes",\n"d": "All of the above"\n},\n"correct": "d"\n},\n"2": {\n"no": "2",\n"mcq": "What is the connection between machine learning and data compression?",\n"options": {\n"a": "Machine learning is used to compress data",\n"b": "Data compression is used to improve machine learning",\n"c": "A system that predicts posterior probabilities can be used for optimal data compression",\n"d": "There is no connection between machine learning and data compression"\n},\n"correct": "c"\n},\n"3": {\n"no": "3",\n"mcq": "What is the main difference between machine learning and data mining?",\n"options": {\n"a": "Machine learning focuses on prediction, while data mining focus

,Question Number,Question,Options,Correct Answer
0,1,What is the primary goal of machine learning?,a. To achieve artificial intelligence | b. To classify data based on models | c. To make predictions for future outcomes | d. All of the above,d
1,2,What is the connection between machine learning and data compression?,a. Machine learning is used to compress data | b. Data compression is used to improve machine learning | c. A system that predicts posterior probabilities can be used for optimal data compression | d. There is no connection between machine learning and data compression,c
2,3,What is the main difference between machine learning and data mining?,"a. Machine learning focuses on prediction, while data mining focuses on discovery | b. Machine learning focuses on discovery, while data mining focuses on prediction | c. Machine learning is used for large datasets, while data mining is used for small datasets | d. Machine learning is used for image compression, while data mining is used for text compression",a
3,4,What is the term used to describe the field that combines machine learning and statistics?,a. Data Science | b. Machine Learning | c. Statistical Learning | d. Artificial Intelligence,c
4,5,What is the term used to describe the process of finding generalisable predictive patterns in machine learning?,a. Generalization | b. Optimization | c. Prediction | d. Classification,a


Quiz saved to quiz.csv


In [48]:
#for saving in logs
log_path=os.path.join(os.getcwd(),"logs")

os.makedirs(log_path,exist_ok=True)
LOG_FILEPATH=os.path.join(log_path,LOG_FILE)


In [52]:
from datetime import datetime
datetime.now().strftime('%d_%m_%Y_%H_%M_%S')
LOG_FILE = f"log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

In [58]:
from mcqgenerator import logger, utils


ModuleNotFoundError: No module named 'mcqgenerator'

In [59]:
import mcqgenerator

ModuleNotFoundError: No module named 'mcqgenerator'

In [ ]:
import sys
print(sys.executable)

c:\Users\Sai Sanjith\mcqgenerator\venv\Scripts\python.exe
